In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd

assert tf.__version__.startswith('2.')

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
PATH = r"D:\data\Q&A\AutoMaster_TrainSet.csv"
data = pd.read_csv(PATH)

In [5]:
data.head()

,QID,Brand,Model,Question,Dialogue,Report
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82943 entries, 0 to 82942
Data columns (total 6 columns):
QID         82943 non-null object
Brand       81642 non-null object
Model       81642 non-null object
Question    82943 non-null object
Dialogue    82941 non-null object
Report      82873 non-null object
dtypes: object(6)
memory usage: 3.8+ MB


In [7]:
data.isnull().sum()

QID            0
Brand       1301
Model       1301
Question       0
Dialogue       2
Report        70
dtype: int64

In [8]:
data.dropna(inplace=True)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81572 entries, 0 to 82942
Data columns (total 6 columns):
QID         81572 non-null object
Brand       81572 non-null object
Model       81572 non-null object
Question    81572 non-null object
Dialogue    81572 non-null object
Report      81572 non-null object
dtypes: object(6)
memory usage: 4.4+ MB


In [10]:
data['Whole_Q'] = data['Brand'] + data['Model'] + data['Question'] + data['Dialogue']

In [11]:
data.head()

,QID,Brand,Model,Question,Dialogue,Report,Whole_Q
0,Q1,奔驰,奔驰GL级,方向机重，助力泵，方向机都换了还是一样,技师说：[语音]|车主说：新的都换了|车主说：助力泵，方向机|技师说：[语音]|车主说：换了...,随时联系,奔驰奔驰GL级方向机重，助力泵，方向机都换了还是一样技师说：[语音]|车主说：新的都换了|车...
1,Q2,奔驰,奔驰M级,奔驰ML500排气凸轮轴调节错误,技师说：你这个有没有电脑检测故障代码。|车主说：有|技师说：发一下|车主说：发动机之前亮故障...,随时联系,奔驰奔驰M级奔驰ML500排气凸轮轴调节错误技师说：你这个有没有电脑检测故障代码。|车主说：...
2,Q3,宝马,宝马X1(进口),2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地换挡位PRND车辆闯...,技师说：你好，4缸自然吸气发动机N46是吧，先挂空档再挂其他档有没有闯动呢，变速箱油液位是否...,行驶没有顿挫的感觉，原地换挡有闯动，刹车踩重没有，这是力的限制的作用，应该没有问题,宝马宝马X1(进口)2010款宝马X1，2011年出厂，2.0排量，通用6L45变速箱，原地...
3,Q4,Jeep,牧马人,3.0V6发动机号在什么位置，有照片最好！,技师说：右侧排气管上方，缸体上靠近变速箱|车主说：[图片]|车主说：是不是这个？|车主说：这...,举起车辆，在左前轮这边的缸体上,Jeep牧马人3.0V6发动机号在什么位置，有照片最好！技师说：右侧排气管上方，缸体上靠近变...
4,Q5,奔驰,奔驰C级,2012款奔驰c180怎么样，维修保养，动力，值得拥有吗,技师说：家庭用车的话，还是可以入手的|技师说：维修保养费用不高|车主说：12年的180市场价...,家庭用车可以入手的，维修保养价格还可以。车况好，价格合理可以入手,奔驰奔驰C级2012款奔驰c180怎么样，维修保养，动力，值得拥有吗技师说：家庭用车的话，还...


In [12]:
def cut(string):
    import jieba
    return ' '.join(jieba.cut(string))

In [13]:
questions = data.Whole_Q

In [14]:
len(questions)

81572

In [15]:
answers = data.Report

In [16]:
len(answers)

81572

In [17]:
questions_cut = ['<start> ' + cut(str(question)) + ' <end>' for question in questions]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MR5E8F~1.WU\AppData\Local\Temp\jieba.cache
Loading model cost 0.756 seconds.
Prefix dict has been built succesfully.


In [18]:
answers_cut = ['<start> ' + cut(str(answer)) + ' <end>' for answer in answers]

In [19]:
questions_cut[0], answers_cut[0]

('<start> 奔驰 奔驰 GL 级 方向机 重 ， 助力 泵 ， 方向机 都 换 了 还是 一样 技师 说 ： [ 语音 ] | 车主 说 ： 新 的 都 换 了 | 车主 说 ： 助力 泵 ， 方向机 | 技师 说 ： [ 语音 ] | 车主 说 ： 换 了 方向机 带 的 有 | 车主 说 ： [ 图片 ] | 技师 说 ： [ 语音 ] | 车主 说 ： 有 助力 就是 重 ， 这车 要 匹配 吧 | 技师 说 ： 不 需要 | 技师 说 ： 你 这 是 更换 的 部件 有 问题 | 车主 说 ： 跑 快 了 还好 点 ， 就 倒车 重 的 很 。 | 技师 说 ： 是 非常 重 吗 | 车主 说 ： 是 的 ， 累人 | 技师 说 ： [ 语音 ] | 车主 说 ： 我 觉得 也 是 ， 可是 车主 是 以前 没 这么 重 ， 选 吧 助理 泵 换 了 不行 ， 又 把 放 向 机换 了 ， 现在 还 这样 就 不 知道 咋 和 车主 解释 。 | 技师 说 ： [ 语音 ] | 技师 说 ： [ 语音 ] <end>',
 '<start> 随时 联系 <end>')

In [20]:
len(answers_cut[0])

19

In [21]:
def max_length(tensor): return max(len(t) for t in tensor)

In [22]:
def tokenize(sent):
    sent_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    sent_tokenizer.fit_on_texts(sent)

    tensor = sent_tokenizer.texts_to_sequences(sent)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, sent_tokenizer

In [23]:
def load_dataset(num_examples, inp, targ):
    inp = inp[:num_examples]
    targ = answers_cut[:num_examples]
    input_tensor, inp_sent_tokenizer = tokenize(inp)
    target_tensor, targ_sent_tokenizer = tokenize(targ)

    return input_tensor, target_tensor, inp_sent_tokenizer, targ_sent_tokenizer

In [24]:
input_tensor, target_tensor, inp_sent, targ_sent = load_dataset(300, questions_cut, answers_cut)

In [25]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [26]:
print(max_length_inp, max_length_targ)
print(len(input_tensor),target_tensor[:2])

1397 141
300 [[  4 514 515   5   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  4 514 515   5   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   

In [27]:
len(input_tensor[0])

1397

In [28]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

240 240 60 60


In [29]:
def convert(sent, tensor):
    for t in tensor:
        if t != 0:
            print("%d ----> %s" % (t, sent.index_word[t]))

In [30]:
print ("Input Sentence; index to word mapping")
convert(inp_sent, input_tensor_train[0])
print ()
print ("Target Sentence; index to word mapping")
convert(targ_sent, target_tensor_train[0])

Input Sentence; index to word mapping
24 ----> <start>
208 ----> 丰田
902 ----> 汉兰达
132 ----> 请问
18 ----> 你
1025 ----> 左
1387 ----> 转向灯
4 ----> ，
18 ----> 你
954 ----> 右
1387 ----> 转向灯
5 ----> 的
411 ----> 速度
63 ----> 要
452 ----> 快
1333 ----> 一半
282 ----> 怎么回事
11 ----> ？
397 ----> 灯泡
1807 ----> 没坏
8 ----> 。
7 ----> 技师
1 ----> 说
2 ----> ：
31 ----> 你好
4 ----> ，
91 ----> 检查一下
397 ----> 灯泡
76 ----> 是不是
372 ----> 烧
10 ----> 了
4 ----> ，
94 ----> 或者
397 ----> 灯泡
2479 ----> 大小
20 ----> 不
130 ----> 一样
73 ----> 大
8 ----> 。
3 ----> |
7 ----> 技师
1 ----> 说
2 ----> ：
397 ----> 灯泡
2479 ----> 大小
20 ----> 不
130 ----> 一样
4 ----> ，
91 ----> 检查一下
60 ----> 吧
4 ----> ，
93 ----> 有没有
110 ----> 维修
78 ----> 过
90 ----> 那个
312 ----> 灯
5 ----> 的
19 ----> 吗
11 ----> ？
3 ----> |
6 ----> 车主
1 ----> 说
2 ----> ：
980 ----> 真的
2480 ----> 一切
29 ----> 都
55 ----> 正常
172 ----> 着
172 ----> 着
852 ----> 放在
534 ----> 那里
4173 ----> 没动
17 ----> 就
71 ----> 出现
50 ----> 这种
44 ----> 故障
10 ----> 了
3 ----> |
6 ----> 车主
1 ----> 说
2 ----> ：
3

In [31]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_sent.word_index) + 1
vocab_tar_size = len(targ_sent.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [32]:
vocab_inp_size, vocab_tar_size

(5099, 1648)

In [33]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 1397]), TensorShape([64, 141]))

In [75]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
#                                        return_sequences=True,
#                                        return_state=True,
                                       recurrent_initializer='glorot_uniform') # Xavier 

    def call(self, x, hidden):
        x = self.embedding(x)
#         output, state = self.gru(x, initial_state = hidden)
#         print(self.gru(x, initial_state = hidden))
        state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [76]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
# sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
state = encoder(example_input_batch, sample_hidden)
# print(output.shape)
# print(state_h.shape)
# print(context_v.shape)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))


Encoder Hidden state shape: (batch size, units) (64, 1024)


In [55]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform') # Xavier 

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
#         output, state_h, context_v = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [56]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
# output, state_h, context_v = encoder(example_input_batch, sample_hidden)
# print(output.shape)
# print(state_h.shape)
# print(context_v.shape)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 1397, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [65]:
class BahdanauAttention(tf.keras.Model):
    # other attention is LuongAttention
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)
        print('query shape:(batch_size, units) {}'.format(query.shape))
        print('values shape:(batch_size, sequence_length, units) {}'.format(values.shape))
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        print('self.W1 shape:(batch_size, units, attention_units) {}'.format(self.W1(values).shape))
        print('self.W2 shape:(batch_size, 1, attention_units) {}'.format(self.W2(hidden_with_time_axis).shape))
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        print('score shape:(batch_size, sequence_length, 1) {}'.format(score.shape))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [66]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

query shape:(batch_size, units) (64, 1024)
values shape:(batch_size, sequence_length, units) (64, 1397, 1024)
self.W1 shape:(batch_size, units, attention_units) (64, 1397, 10)
self.W2 shape:(batch_size, 1, attention_units) (64, 1, 10)
score shape:(batch_size, sequence_length, 1) (64, 1397, 1)
(64, 1397, 1)
Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 1397, 1)


In [36]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [38]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 1608)


In [39]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [40]:
checkpoint_dir = './'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [43]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_sent.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

            loss += loss_function(targ[:,t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:,t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [ ]:
import warnings
warnings.filterwarnings('ignore')

EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

...

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

    def call(self, query, values):
        # query: hidden
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
#         print(attention_weights.shape)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, at

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence_english(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
def query(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))